# **Runnables**

Let's now learn how to put multiple chains together in an organized way.

## **What are `RunnableSequence`?**

Observe that all the chains are RunnableSequence.

LangChain implements the RunnableInterface, which allows the composition or chaining of various components into a RunnableSequence.  

**What is Runnable?**  
An object with standard methods like invoke, stream, batch, etc... You can create a Runnable using the **RunnableLambda** class in LangChain. RunnableLambda is a LangChain abstraction that allows Python-callable functions to be transformed into functions compatible with LangChain's pipeline operations.

**What is RunnableSequence?**  
You can compose these Runnable objects together to create a pipeline of operations.

**What is RunnableParallel?**  
Runnable that runs a mapping of Runnables in parallel, and returns a mapping of their outputs.

RunnableParallel is one of the two main composition primitives for the LCEL, alongside RunnableSequence. It invokes Runnables concurrently, providing the same input to each.

In [1]:
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableParallel

def sum_method(x: int) -> int:
    return x + x

def multiply_method(x: int) -> int:
    return x * x

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Converting methods to Runnables

runnable_1 = RunnableLambda(lambda x: sum_method(x))
runnable_2 = RunnableLambda(lambda x: multiply_method(x))

In [3]:
# Defining a Runnable Sequence

runnable_sequence = RunnableSequence(first=runnable_1, last=runnable_2)

runnable_sequence.invoke(2)

16

In [4]:
# Another way to define Runnable Sequence

runnable_sequence = runnable_1 | runnable_2

runnable_sequence.invoke(3)

36

In [5]:
# Defining a Runnable Parallel

runnable_parallel = RunnableParallel(sum_out=runnable_1, mult_out=runnable_2)

runnable_parallel.invoke(2)

{'sum_out': 4, 'mult_out': 4}

In [6]:
# Another way to define Runnable Parallel

runnable_parallel = RunnableParallel({'sum_out': runnable_1, 'mult_out': runnable_2})

runnable_parallel.invoke(3)

{'sum_out': 6, 'mult_out': 9}

## **Example: Putting Multiple Runnable in RunnableSequence**

In [7]:
from datetime import datetime

from langchain_core.runnables import RunnableLambda, RunnableSequence


# Define the transformations as simple functions
def greet(name):
   return f"Hello, {name}!"


def append_datetime(text):
   current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
   return f"{text} The current date and time is {current_datetime}"


def to_uppercase(text):
   return text.upper()


def add_exclamation(text):
   return f"{text}!"


# Wrap the functions in RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))
datetime_runnable = RunnableLambda(lambda x: append_datetime(x))
uppercase_runnable = RunnableLambda(lambda x: to_uppercase(x))
exclamation_runnable = RunnableLambda(lambda x: add_exclamation(x))


# Create a RunnableSequence with the wrapped runnables
chain = RunnableSequence(
   first=greet_runnable,
   middle=[datetime_runnable, uppercase_runnable],
   last=exclamation_runnable,
)


# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

HELLO, ALICE! THE CURRENT DATE AND TIME IS 2025-02-28 15:59:43!


In [8]:
# Create a RunnableSequence with the wrapped runnables
chain = greet_runnable | datetime_runnable | uppercase_runnable | exclamation_runnable

# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

HELLO, ALICE! THE CURRENT DATE AND TIME IS 2025-02-28 15:59:47!


## **RunnablePassThrough**

Runnable to passthrough inputs either:
- unchanged (or)
-  with additional keys - with the help of assign method.

This Runnable behaves almost like the identity function, except that it can be configured to add additional keys to the output, if the input is a dict.

In [9]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1)

{'origin': 1, 'modified': 2}

In [10]:
# Modified Example

runnable = {"original": RunnablePassthrough()} | RunnableLambda(lambda x: x["original"]+1)

runnable.invoke(1)

2

**Important:**  
In some cases, it may be useful to pass the input through while adding some keys to the output. In this case, you can use the assign method:

In [11]:
from langchain_core.runnables import RunnablePassthrough

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')

{'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

## **Case Study**

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [13]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [14]:
WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in requests about the topics and write organized research reports on those topics.
Also you share the appropriate references at the end of report."""

HUMAN_PROMPT_1 = """Write an organized research report about {topic}."""

writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_1)
])


writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

writer_chain = writer_chat_template | writer_chat_model | output_parser

# research_report = writer_chain.invoke({"topic": "how transformers algorithm works?"})

# print(research_report)

In [15]:
REVIEWER_SYS_PROMPT = """You are a reviewer for research reports. 
You take in research reports and provide a feedback on them."""

HUMAN_PROMPT_2 = """Provide feedback as 5 concise bullet points on this research report: 

{report}"""

reviewer_chat_template = ChatPromptTemplate.from_messages([
    ("system", REVIEWER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_2)
])

reviewer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                                 model="gpt-4o-mini", 
                                 temperature=0.2)

reviewer_chain = reviewer_chat_template | reviewer_chat_model | output_parser

# report_feedback = reviewer_chain.invoke({"report": research_report})

# print(report_feedback)

In [16]:
FINAL_WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in a research report in a set of bullet points with feedback to improve.
You revise the research report based on the feedback and write a final version."""

HUMAN_PROMPT_3 = """Write a reviewed and improved version of research report: 

{report}

based on this feedback:

{feedback}"""

final_writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", FINAL_WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_3)
])


final_writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

final_writer_chain = final_writer_chat_template | final_writer_chat_model | output_parser

# final_report = final_writer_chain.invoke({"report": research_report, "feedback": report_feedback})

# print(final_report)

## **What are these chains?**

Given that we have created three chains above, let's now analyse what these chains are?

In [17]:
print(type(writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [18]:
print(type(reviewer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [19]:
print(type(final_writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


## **Composing the Chains together**

**RunnablePassthrough** for passing data unchanged from previous steps for use as input in later steps.

<img src="images/composing_chains.jpg">

In [20]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = {"report" : writer_chain} | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain

final_report = composed_chain.invoke({"topic": "What are Runnables in LangChain?"})

In [21]:
# Or we can use the following code as well
# composed_chain = RunnablePassthrough().assign(report=writer_chain) | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain

In [22]:
from IPython.display import Markdown

Markdown(final_report)

# Research Report: Runnables in LangChain

## Introduction
LangChain is an innovative framework designed to facilitate the development of applications that utilize language models. A key component of LangChain is the concept of "Runnables," which serve as fundamental building blocks within the framework. Runnables enable developers to create modular and reusable components that can execute various tasks related to language processing. This report provides a comprehensive overview of Runnables in LangChain, including their definition, functionality, use cases, advantages, and future directions.

## Definition of Runnables
Runnables in LangChain are abstractions that encapsulate specific tasks or operations that can be executed independently or in conjunction with other Runnables. Each Runnable is a self-contained unit of work that can take inputs, perform computations or transformations, and produce outputs. This modular approach allows developers to build complex workflows by chaining together multiple Runnables, enhancing the overall efficiency and organization of the application.

## Functionality of Runnables
Runnables in LangChain are designed to be flexible and versatile, supporting a variety of tasks, including but not limited to:

1. **Text Generation**: Runnables can invoke language models to generate text based on given prompts. For example, a Runnable could be created to generate a summary of a document by passing the document text as input to a language model.
   
   ```python
   from langchain import Runnable
   from langchain.llms import OpenAI

   text_generator = Runnable(OpenAI(model="text-davinci-003"))
   summary = text_generator.run("Summarize the following text: [insert text here]")
   ```

2. **Data Transformation**: Runnables can process and transform data, such as cleaning or formatting text. For instance, a Runnable could be implemented to remove special characters from a dataset.

3. **API Calls**: Runnables can be used to make API requests to external services, integrating additional functionalities into the application. For example, a Runnable could fetch data from a weather API and format it for user display.

4. **Conditional Logic**: Developers can implement conditional logic within Runnables to control the flow of execution based on specific criteria, allowing for dynamic responses in applications.

## Use Cases
Runnables can be employed in various scenarios, including:

1. **Chatbots**: Runnables can handle user inputs, generate responses, and manage conversation flows. For example, a chatbot could use Runnables to process user queries and provide relevant answers.

2. **Data Pipelines**: In data processing applications, Runnables can be chained together to create a pipeline that ingests, processes, and outputs data. This is particularly useful in ETL (Extract, Transform, Load) processes.

3. **Content Creation**: Runnables can automate the generation of articles, summaries, or reports based on user-defined parameters, streamlining content production.

4. **Interactive Applications**: Runnables can facilitate interactive applications that require real-time processing and feedback, such as educational tools or gaming applications.

## Advantages of Runnables
The use of Runnables in LangChain offers several advantages:

1. **Modularity**: Runnables promote a modular design, allowing developers to create reusable components that can be easily integrated into different applications.

2. **Scalability**: The ability to chain Runnables together enables the development of scalable applications that can handle complex workflows efficiently.

3. **Maintainability**: By encapsulating specific tasks within Runnables, the codebase becomes easier to maintain and update, reducing technical debt.

4. **Flexibility**: Runnables can be easily modified or replaced without affecting the overall application, providing flexibility in development.

## Comparative Analysis
When compared to similar concepts in other frameworks, Runnables in LangChain stand out due to their emphasis on modularity and ease of integration. For instance, while other frameworks may offer monolithic components, Runnables allow for granular control over individual tasks, making it easier to adapt and extend applications. This modular approach not only enhances reusability but also simplifies debugging and testing processes.

## Future Directions
As LangChain continues to evolve, there are several potential developments for Runnables that could enhance their functionality:

1. **Enhanced Error Handling**: Future versions could introduce more robust error handling mechanisms within Runnables, allowing for better fault tolerance in complex workflows.

2. **Integration with Emerging Technologies**: Runnables could be expanded to integrate with emerging technologies such as real-time data streams or advanced machine learning models, broadening their applicability.

3. **User-Friendly Interfaces**: Developing graphical interfaces for creating and managing Runnables could make the framework more accessible to non-technical users, fostering wider adoption.

4. **Community Contributions**: Encouraging community contributions to the library of Runnables could lead to a richer ecosystem of pre-built components, accelerating development for users.

## Conclusion
Runnables are a core feature of the LangChain framework, providing a powerful mechanism for building modular and reusable components in language model applications. Their versatility allows developers to create a wide range of applications, from chatbots to data processing pipelines. By leveraging Runnables, developers can enhance the scalability, maintainability, and flexibility of their projects, paving the way for innovative solutions in the realm of language processing.

## References
1. LangChain Documentation. (2023). Retrieved from [LangChain Documentation](https://langchain.readthedocs.io/en/latest/)
2. LangChain GitHub Repository. (2023). Retrieved from [LangChain GitHub](https://github.com/hwchase17/langchain)
3. Research Papers on Language Models and Frameworks. (2023). Various authors.